In [1]:
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import cv2
import numpy as np
import os
import joblib

In [2]:
# 1. 数据准备
train_path = "Dataset"  # 训练数据集路径
test_path = "Dataset"   # 测试数据集路径
train_images = []       # 存储训练图像的灰度图像列表
train_labels = []       # 存储训练图像的标签列表
test_images = []        # 存储测试图像的灰度图像列表
test_labels = []        # 存储测试图像的标签列表

# 遍历训练数据集的每个类别目录
for category in os.listdir(train_path):
    # 对每个类别的前150张图像进行处理
    for filename in os.listdir(os.path.join(train_path, category))[:150]:
        # 读取图像并转换为灰度图像
        img = cv2.imread(os.path.join(train_path, category, filename))
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # 将灰度图像和对应的类别标签添加到训练集列表中
        train_images.append(img_gray)
        train_labels.append(category)

    # 对每个类别的后面的图像（超过150张）进行处理，作为测试集
    for filename in os.listdir(os.path.join(train_path, category))[150:]:
        # 读取图像并转换为灰度图像
        img = cv2.imread(os.path.join(train_path, category, filename))
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # 将灰度图像和对应的类别标签添加到测试集列表中
        test_images.append(img_gray)
        test_labels.append(category)

In [12]:
# 2. 特征提取和表示（使用空间金字塔）
def extract_features_with_pyramid(images):
    sift = cv2.SIFT_create()  # 创建SIFT特征提取器
    descriptors = []          # 存储图像的特征描述符列表
    for img in images:
        pyramid_descriptors = []  # 存储图像的金字塔特征描述符列表
        for level in range(3):   # 三级空间金字塔
            scale = 2 ** level    # 计算金字塔缩放比例
            # 将图像按照缩放比例进行缩放
            resized_img = cv2.resize(img, (img.shape[1] // scale, img.shape[0] // scale))
            # 使用SIFT提取缩放后图像的关键点和特征描述符
            kp, des = sift.detectAndCompute(resized_img, None)
            # 如果存在特征描述符，则将其添加到金字塔特征描述符列表中
            if des is not None:
                pyramid_descriptors.extend(des)
        # 将金字塔特征描述符列表添加到总的特征描述符列表中
        descriptors.append(pyramid_descriptors)
    return descriptors

# 对训练集和测试集图像提取空间金字塔特征描述符
train_descriptors = extract_features_with_pyramid(train_images)
test_descriptors = extract_features_with_pyramid(test_images)

In [13]:
# 3. 特征聚类
all_descriptors = [desc for sublist in train_descriptors for desc in sublist]
kmeans = KMeans(n_clusters=100)
kmeans.fit(all_descriptors)
visual_words = kmeans.cluster_centers_

In [14]:
def build_features(descriptors, kmeans):
    """
    构建图像词袋表示
    """
    features = []  # 存储图像的词袋表示列表
    
    # 遍历每个图像的特征描述符列表
    for desc in descriptors:
        histogram = np.zeros(len(kmeans.cluster_centers_), dtype=np.float32)  # 初始化特征直方图向量
        
        if desc is not None:
            # 使用K均值模型对特征描述符进行聚类，获取对应的类别标签
            labels = kmeans.predict(desc)
            # 统计每个类别标签的出现次数，构建特征直方图
            for label in labels:
                histogram[label] += 1

        # 将特征直方图添加到图像的词袋表示列表中
        features.append(histogram)
    
    return features

# 构建训练集和测试集的图像词袋表示
train_features = build_features(train_descriptors, kmeans)
test_features = build_features(test_descriptors, kmeans)


In [15]:
# 5. 训练分类器
svm = SVC()
svm.fit(train_features, train_labels)

# 6. 保存模型参数
# joblib.dump(svm, 'pkl2/svm_model.pkl')
# joblib.dump(kmeans, 'pkl2/kmeans_model.pkl')


SVC()

In [16]:
# 7. 评估分类器
predictions = svm.predict(test_features)
accuracy = accuracy_score(test_labels, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.5360178970917227
